## MACs

### MACsを計算する


In [18]:
import torch
from torch import nn
import thop

In [8]:
def calc_output_shape(layer, input_shape):
    input = torch.randn(input_shape)
    output = layer(input)
    return tuple(output.shape)

In [14]:
# 畳み込み層のMACsを計算する関数を作る

def calc_macs_conv2d(layer, input_shape):
    batch_size, in_ch, in_h, in_w = input_shape
    # フィルタのサイズ
    out_ch, in_ch, f_h, f_w = layer.weight.shape
    
    # 出力サイズ
    _, _, out_h, out_w = calc_output_shape(layer, input_shape)
    
    # macs
    macs = in_ch * out_ch * f_h * f_w * out_h * out_w
    
    return macs


# 全結合層のMACsを計算する関数を作る
def calc_macs_fc(layer, input_shape):
    batch_size, in_features = input_shape
    out_features = layer.weight.shape[0]
    
    # macs
    macs = batch_size * in_features * out_features
    
    return macs

In [15]:
calc_macs_conv2d(layer, input_shape)

1143072

In [16]:
linear_layer = nn.Linear(64, 10)
input_shape = (1, 64)
calc_macs_fc(linear_layer, input_shape)

640

In [6]:
input_shape = (1, 1, 128, 128)
x = torch.randn(input_shape)
layer = nn.Conv2d(1, 8, kernel_size=3)
output = layer(x)

In [7]:
output.shape

torch.Size([1, 8, 126, 126])

In [9]:
calc_output_shape(layer, input_shape)

(1, 8, 126, 126)

## thopでMACsを求める

In [23]:
# MACsを計算する
macs, params = thop.profile(layer, (x, ))

[INFO] Register count_convNd() for <class 'torch.nn.modules.conv.Conv2d'>.


In [25]:
thop.profile(linear_layer, torch.randn(input_shape))

[INFO] Register count_linear() for <class 'torch.nn.modules.linear.Linear'>.


(640.0, 0)